<a href="https://colab.research.google.com/github/jefersonpython/Jeferson/blob/main/Carteira_de_Op%C3%A7oes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas
!pip install pandas_datareader
!pip install numpy
!pip install yfinance
!pip install math
!pip intall scipy.stats

ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
ERROR: unknown command "intall" - maybe you meant "install"


In [5]:
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from dateutil.relativedelta import relativedelta
from math import log, sqrt, pi, exp
from scipy.stats import norm
import yfinance as yf

def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))    

def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"    

stock = 'PETR4.SA'
expiry = '2022-02-18'
strike_price = 32.01

today = datetime.timestamp(datetime.now())
one_year_ago = datetime.timestamp(datetime.now() - relativedelta(years=1))

df = yf.download(stock, start='2022-01-01',end ='2022-02-06')
df = pd.DataFrame(df)

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)
sigma = np.sqrt(252) * df['returns'].std()

# SELIC
uty = 10.65
    
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365

S = lcp # fechamento anterior do papel
K = strike_price # Strike
T = t # tempo
r = uty # taxa de juros
sigma = sigma # volatilidade

print('***** DADOS DE ENTRADA *****')
print('Ativo',stock)
print('Preço', S)
print('Strike',K)
print('Vencimento',expiry)
print('Dias até o vencimento', T)
print('Volatilidade', sigma)
print('Selic',r)

print('\n***** DADOS DE SAÍDA *****')
print('The Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))
a = call_delta(S,K,T,r,sigma)
b = call_gamma(S,K,T,r,sigma)
c = call_vega(S,K,T,r,sigma)
d = call_theta(S,K,T,r,sigma)
e = call_rho(S,K,T,r,sigma)
print('delta',a,'\ngamma',b,'\nvega',c,'\ntheta',d,'\nsigma',e)

[*********************100%***********************]  1 of 1 completed
***** DADOS DE ENTRADA *****
Ativo PETR4.SA
Preço 32.630001068115234
Strike 32.01
Vencimento 2022-02-18
Dias até o vencimento 0.005479452054794521
Volatilidade 0.3173650185252256
Selic 10.65

***** DADOS DE SAÍDA *****
The Option Price is:  2.4346163363100395
delta 0.9995374782167168 
gamma 0.002156997634694313 
vega 3.993742235348248e-05 
theta -3.2153577368447985 
sigma 0.0016537146655080897
